In [1]:
from database import Database
from table import Table
from attribute import Attribute
from TableOperators import *

In [2]:
def test_data():
    A = Attribute("A", "integer")
    B = Attribute("B", "integer")
    C = Attribute("C", "string")
    D = Attribute("D", "string")
    E = Attribute("E", "integer")
    F = Attribute("F", "string")
    G = Attribute("G", "string")

    new_table1 = Table("Student", [A,B,C])
    new_table2 = Table("Grade", [E, G])
    new_table3 = Table("test3", [A,B,C])

    new_table1.add_fd("A->B")
    new_table1.add_fd("A->C")
    new_table1.master_key = "A"
    new_table1.add_tuple((1,18,"Tony"))
    new_table1.add_tuple((2,22,"Tom"))
    new_table1.add_tuple((3,21,"Alex"))
    new_table1.add_tuple((4,24,"Matt"))

    new_table2.add_fd("E->G")
    new_table2.master_key = "E"
    new_table2.add_tuple((10,"A"))
    new_table2.add_tuple((12,"B"))

    new_table3.add_fd("AB->C")
    new_table3.master_key = "AB"
    new_table3.add_tuple((1,10,"A"))
    new_table3.add_tuple((1,12,"B"))

    
    db = Database()
    db.add_table(new_table1)
    db.add_table(new_table2)
    db.add_table(new_table3)

    return db

In [3]:
print(test_data())

This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This breaks the consistency implied by the FD: A->B
This breaks the consistency implied by the FD: A->B
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 22 | Tom | 
Table: Grade
 | E | G |
----------
 | 10 | A | 
 | 12 | B | 
Table: test3
 | A | B | C |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | B | 



In [25]:
def create_tables():
    table_name = input("Please input new table name:")
    attrs = []
    print("Please input at least one attribute. Input 'quit' at any time to stop.")

    while True:
        input_str = input("Please input the name and type (\'integer\'  or \'string\') for one attribute. Use \',\' as delimiter :")
        if input_str == "quit":
            if len(attrs) == 0:
                print("You need to input at least 1 attribute for the new table")
            else:
                break
        else:
            input_split = input_str.replace(" ","").split(",")
            if len(input_split) != 2 or input_split[1] == "":
                print("The input is not correct, please input again.")
            else:
                attrs.append(Attribute(input_split[0], input_split[1]))
                print("Successfully add a new attribute")

    new_table = Table(table_name, attrs)
    print("The tabel " + table_name + " have created successfully!")
    print(new_table)

    return new_table


def create_constraint(table, db):
    while True:
        input_str = input("Please input Boolean condition :")
        if input_str == "quit":
            break
        feedback = table.add_boolean_conditions(input_str.replace(" ",""))
        print(feedback)

    while True:

        input_str = input("Please input FD:")
        if input_str == "quit":
            break
        feedback = table.add_fd(input_str.replace(" ",""))
        print(feedback)

    while True:
        if len(table.fds) == 0:
            break
        else:
            table.print_fds()
        input_str = input("Do you want to remove some Fds(yes or no):")
        if input_str == "no":
            break
        else:
            input_str = input("Please input a Fd that you want to remove:")
            feedback = table.remove_fd(input_str.replace(" ",""))
            print(feedback)

    while True:

        input_str = input("Please input MVD:")
        if input_str == "quit":
            break

        feedback = table.add_mvd(input_str.replace(" ",""))
        print(feedback)

#     while True:

#         input_str = input("Please input foreign key and its table(use , as delimiter):")
#         if input_str == "quit":
#             break
#         input_split = input_str.replace(" ","").split(",")
#         if len(input_split) != 2 or input_split[1] == "":
#             print("Invalid input, please input again!")
#             continue
#         if input_split[1] not in db.tables:
#             print("There is no such table!")
#             continue
#         else:
#             foregin_table = db.tables[input_split[1]]

#         result = foregin_table.get_keys().pop()
#         if not input_split[0] in result:
#             print("The key is not the key in the table " + input_split[1] + ".")
#             continue

#         table.add_foreign_key(input_str[0], foregin_table)
#         print("Add foreign key successfully")

    return table


def update_normal_form(db, table):
    if table.nf == "":
        table.get_normal_form()

    print("Currently this table [" + table.name + "] has normal form: " + table.nf)

    if not (table.nf == "3NF" or table.nf == "BCNF"):
        print("You can either delete this table or update its FD's.")
        decision = input("Either type \'del\' for deletion of the table or \'u\' to update:")

        if decision == 'del':
            input_str = input("Are you sure delete the table "+ table.name +"?(yea or no)")
            if input_str == "yes":
                db.delete_table(table.name)
                print("delete the tabel [" + table.name + "] successfuly")
                table = None
            else:
                update_normal_form(db, table)
        elif decision == 'u':
            done = False
            while not done:
                print("Current FD(s) in the table: " + str(table.fds) + "\n")
                input_str = input("Type \'a\' to add a new FD or \'d\' to delete a FD: ")
                if input_str == 'a':
                    new_fd = input("Add a new FD: ")
                    feedback = table.add_fd(new_fd.replace(" ",""))
                    print(feedback)
                    table.get_normal_form()
                    again = input("The table now has normal form " + table.nf + " would you like to continue (y/n)?")
                    if again == 'n':
                        done = True

                elif input_str == 'd':
                    del_fd = input("Enter FD to be deleted: ")
                    feedback = table.remove_fd(del_fd.replace(" ",""))
                    print(feedback)
                    if len(table.fds) == 0:
                        print("The table has no FD left! Please start again")
                        break

                    else:
                        table.get_normal_form()
                        again = input("The table now has normal form " + table.nf + " would you like to continue (y/n)?")
                        if again == 'n':
                            done = True
                else:
                    print("That's not a valid decision!")

        else:
            print("That's not a valid decision!")

    return table

def user_define_key(table):
    table.get_keys()
    print("Please define keys for the table [" + table.name + "].")
    table.user_define_key()

In [ ]:
db = Database()
# Task I.a
# Define new tables
while True:
    new_table = create_tables()
    db.add_table(new_table)
    is_add_new_table = input("Do you still want to define a new table(yes or no)?")
    if is_add_new_table == "no":
        print('Current tables in the Database:\n')
        print(db.tables)
        break

In [12]:
# Task I.b
# Ask users to input possible constraints for each defined table
for table_name, table in db.tables.items():
    print("Please input constraint for the table [" + table.name + "]. Input 'quit' at any time to stop.")
    table = create_constraint(table, db)

Please input constraint for the table [Student]. Input 'quit' at any time to stop.
Please input Boolean condition :B>18
Add boolean conditions successfully
Please input Boolean condition :quit
Please input FD:A->BC
Added a new fd successfully: A->BC
Please input FD:quit
The table named:  Student 
Has FD's:  ['A->B', 'A->C']
Do you want to remove some Fds(yes or no):no
Please input MVD:quit
Please input constraint for the table [Grade]. Input 'quit' at any time to stop.
Please input Boolean condition :quit
Please input FD:quit
Please input MVD:quit


In [13]:
# Task I.c
# Evaluate the NF category for the user defined DB
for table_name, table in db.tables.copy().items():
    table = update_normal_form(db, table)
    if table != None:
        db.tables[table_name] = table

This table has normal form: BCNF
Currently this table [Student] has normal form: BCNF
This table has normal form: BCNF
Currently this table [Grade] has normal form: BCNF


In [10]:
# Task I.d
# Ask users to define keys for each table
for table_name, table in db.tables.items():
    table.print_keys()
    table.user_define_key()
    print('Attributes in the table: ' + repr(table.attributes_names))
    #print(table.attributes)

All keys for this table are :  {'A'}
What key would you like to use? (type 'rand' to have it selected for you): A
Great selection! A is a viable key
Attributes in the table: {'C', 'B', 'A'}
All keys for this table are :  {'C'}
What key would you like to use? (type 'rand' to have it selected for you): C
Great selection! C is a viable key
Attributes in the table: {'C', 'E', 'D'}


In [3]:
# Task II.a
# Input new tuples to all tables
db = test_data()
while True:
    print("The tables in current db: \n")
    print(db)
    input_str = input("Please choose a table in order to input a new tuple into it(or quit to terminate): ")
    if input_str == 'quit':
        break
    if input_str in db.tables:
        table_to_add = db.tables[input_str]
        while True:
            in_tuple_str = input("Input a new tuple into " + input_str + " using \',\' as delimiter(or quit):")
            if in_tuple_str == "quit":
                break
            in_tuple_as_list = list(in_tuple_str.split(','))
            type_list = [a.type for a in table_to_add.attributes]
            for i, el in enumerate(in_tuple_as_list):
                if type_list[i] == "integer":
                    in_tuple_as_list[i] = int(el)
            # handle case of integer
            table_to_add.add_tuple(tuple(in_tuple_as_list))

    else:
        print("Invaild input, please input again!")

This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This breaks the consistency implied by the FD: A->B
This breaks the consistency implied by the FD: A->B
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
The tables in current db: 

Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 22 | Tom | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | B | 

Please choose a table in order to input a new tuple into it(or quit to terminate): Grade
Input a new tuple into Grade using ',' as delimiter(or quit):2,4,"A"
Input a new tuple into Grade using ',' as delimiter(or quit):3,10,A
This breaks the consistency implied by the FD: AE->G
Input a new tuple into Grade using ',' as delimiter(or quit):3,10,"A"
This breaks the consistency implied by the FD: AE->G
Input a new tuple into Grade using ',' as delimiter(or

In [6]:
# Task II.b
# Delete a tuple based on key value
db = test_data()
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to delete a tuple from any table? (y/n)")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to delete from?")
        table = db.tables[table_name]
        print("This the table you will be deleting from: \n", table)
        print("Which has the key: ", table.keys)
        key = input("Please provide the key for the tuple you'd like to delete: ")
        table.remove_tuple(key)


This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This breaks the consistency implied by the FD: A->B
This breaks the consistency implied by the FD: A->B
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 22 | Tom | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | B | 

Would you like to delete a tuple from any table? (y/n)y
Which table would you like to delete from?Student
This the table you will be deleting from: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 22 | Tom | 

Which has the key:  set()
Please provide the key for the tuple you'd like to delete: 2
The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
Table: Grade
 | A | E | G |
-----------

In [ ]:
# Task II.c
# Find tuples based on conditions
db = test_data()
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to find any tuples from some conditions in the tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to search in?")
        table = db.tables[table_name]
        attr_name = input("Which attribute would you like to condition by?")
        values = input("Which values would you like to condition on?")
        conditions = input("Which condition do you want (>, >, <>, or ==)")
        print(table.get_tuple_conditions(attr_name, values, conditions))

# Group by attribute(s) in specific table
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to group by any attributes from the tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to group by in?")
        table = db.tables[table_name]
        attr_name = input("Which attribute would you like to group by?")
        table.group_by(attr_name)

This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This breaks the consistency implied by the FD: A->B
This breaks the consistency implied by the FD: A->B
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 22 | Tom | 
Table: Grade
 | E | G |
----------
 | 10 | A | 
 | 12 | B | 
Table: test3
 | A | B | C |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | B | 

Would you like to find any tuples from some conditions in the tables (y/n)?y
Which table would you like to search in?Grade
Which attribute would you like to condition by?E
Which values would you like to condition on?11
Which condition do you want (>, >, <>, or ==)>
{'> 1

In [16]:
# TODO Task II.d
# Choose to perform the operators for two tables
db = test_data()
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to perform operators on tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_one = input("Select the first table: ")
        table1 = db.tables[table_one]
        table_two = input("Select the second table: ")
        table2 = db.tables[table_two]
        while True:
            input_str = input('Select operators:\
            \n \'cjoin\' for cross join\
                              \n \'njoin\' for natural join\
                              \n \'union\' for union\
                              \n \'inter\' for intersection\
                              \n \'diff\' for difference\n')
            if input_str == 'njoin':
                print(njoin_table(table1,table2))
            if input_str == 'cjoin':
                print(cjoin_table(table1,table2))
            if input_str == 'union':
                print(union(table1,table2))
            if input_str == 'inter':
                print(intersection(table1,table2))
            if input_str == 'diff':
                print(difference(table1,table2))
            else:
                break

This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This breaks the consistency implied by the FD: A->B
This breaks the consistency implied by the FD: A->B
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 22 | Tom | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | B | 
Table: test3
 | A | B | C |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | B | 

Would you like to perform operators on tables (y/n)?y
Select the first table: Student
Select the second table: test3
Select operators:            
 'cjoin' for cross join                              
 'njoin' for natural join                            

In [23]:
# TODO Task II.e
# Delete a table and ensure the across-table integrity
db = test_data()
print([name for name, _ in db.tables.items()])
table_to_del = input("What is the name of the table you'd like to delete?")
db.delete_table(table_to_del)

This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This breaks the consistency implied by the FD: A->B
This breaks the consistency implied by the FD: A->B
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
This table is not part of a database! No foreign key to check
['Student', 'Grade', 'test3']
What is the name of the table you'd like to delete?Grade


True

In [24]:
# Write database to text file
write_ans = input("Would you like to write whole database (\'all\') or a specific table to a txt file?")
if write_ans == "all":
    db.write_database()
elif write_ans == 'n':
    print("Nothing")
else:
    if write_ans in db.tables:
        db.write_table(write_ans)
    else:
        print("There is not table with name: " + write_ans)

Would you like to write whole database ('all') or a specific table to a txt file?all
